In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)

In [19]:
ll lightning_logs/version_2/checkpoints/

total 131144
-rw-rw-r-- 1 ubuntu 134289692 Jan  3 20:34 'epoch=95-step=767.ckpt'


In [20]:
checkpoint_path = './lightning_logs/version_2/checkpoints/epoch=95-step=767.ckpt'

In [21]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [22]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [23]:
dataloader = DataLoader(dataset, batch_size=10, num_workers=2)

In [24]:
videos, questions, answers, hidden_states, _  = iter(dataloader).next()

In [25]:
predictions = model.eval()(videos)
predictions

tensor([[0.8450, 0.5448, 0.9398],
        [0.1780, 0.5085, 0.1194],
        [0.2152, 0.5859, 0.5385],
        [0.7205, 0.5345, 0.5399],
        [0.0955, 0.3855, 0.8686],
        [0.1475, 0.6312, 0.9583],
        [0.8030, 0.5446, 0.8962],
        [0.5937, 0.5861, 0.6022],
        [0.6369, 0.5447, 0.6521],
        [0.4800, 0.4428, 0.6196]], grad_fn=<AddmmBackward>)

In [26]:
hidden_states

tensor([[0.8956, 0.3618, 0.8750],
        [0.2068, 0.9950, 0.1250],
        [0.2169, 0.6080, 0.5417],
        [0.6988, 0.5578, 0.5000],
        [0.0482, 0.1709, 0.9167],
        [0.0823, 0.5678, 0.9583],
        [0.8293, 0.9246, 0.8750],
        [0.5823, 0.0050, 0.5000],
        [0.6506, 0.9899, 0.5417],
        [0.4317, 0.1307, 0.5833]], dtype=torch.float64)

In [18]:
mse_loss = torch.nn.MSELoss(reduction='sum')
mse_hidden = mse_loss(predictions[0:2,:].type(torch.float32),
                      hidden_states[0:2,:].type(torch.float32))
mse_hidden

tensor(0.2628, grad_fn=<MseLossBackward>)

In [33]:
((772.6472 - 543.4424)**2 + (912.8349-996.8575)**2 + (36.0000-37.2697)**2 + \
(627.9586-543.4633)**2 + (1077.1292-996.8958)**2 + (25.0000-37.2711)**2)

73323.68388475002

In [ ]:
model.eval()(videos)

In [22]:
predictions[0:2,:]

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711]], grad_fn=<SliceBackward>)